In [18]:
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import itertools
from tqdm import tqdm
import tqdm.contrib.itertools as tqdm_it

# image sources: https://www.easyhdr.com/examples/

In [19]:
def show(img):
    i = img.astype('uint8')*255 if img.dtype == 'bool' else img
    # print(i.dtype, i.ndim)
    plt.figure(figsize=(10, 10))
    if img.ndim == 3:
        # color
        plt.imshow(img[..., ::-1], interpolation='none')
    else:
        plt.imshow(img, cmap='gray', interpolation='none')
        # plt.imshow(img, cmap='gray', vmin=0, vmax=255, interpolation='none')
    plt.show()

## Read Image

In [20]:
# folder = 'cannon'
# folder = 'castle'
folder = 'cs_1'
type = "TIME"
# type = "EV"

folder = "data/"+folder

In [21]:
file_names = os.listdir(folder)
imgs = [(cv2.imread(folder + "/" + f), np.float32(f.rsplit(".", 1)[0])) for f in file_names]
imgs, evs = zip(*imgs)

shape = np.array(imgs[0].shape)[:2][::-1]//4
imgs = [cv2.resize(i, tuple(shape)) for i in imgs]

imgs = np.array(imgs)
# ev = 0 for 1 sec
if type == "TIME":
    dt = evs
else:
    dt = 2**(-np.array(evs))
lndt = np.log(dt)
gimgs = [cv2.cvtColor(i, cv2.COLOR_RGB2GRAY) for i in imgs]
print(f"Loading {len(imgs)} images from folder \"{folder}\", dt:")
print(dt)
print("ev:", evs)

Loading 7 images from folder "data/cs_1", dt:
(0.25, 0.5, 1.0, 15.0, 2.0, 4.0, 8.0)
ev: (0.25, 0.5, 1.0, 15.0, 2.0, 4.0, 8.0)


# Alignment

## MTB (TODO)

In [22]:
def mtb(img, threshold = 20):
    med = np.median(img)
    # bimg = np.zeros_like(img, dtype=bool)
    bimg = img >= (med)
    eximg = np.abs(img.astype(int)-med) < threshold
    # low   | low_thre  | high_thre | high
    # F     | F         | T         | T
    # F     | T         | T         | F
    return bimg, eximg

In [23]:
threshold = 20
bimgs, eximgs = zip(*[mtb(i, threshold) for i in gimgs])

# Recovering Response Curve

In [24]:
def SolveResponseCurve(imgs, P, N, z_mid):
    print(f"Z_mid = {z_mid}, P = {P}, N = {N}")

    # pixs[P][N]
    fimgs = [f.flatten() for f in imgs]
    idx = np.random.choice(fimgs[0].shape[0], N, replace=False)
    pixs = np.array([f[idx] for f in fimgs])
    shape = (N*P+255, 256+N)

    # =====================================
    # Ax = b
    # mat A
    A = np.zeros(shape, dtype=float)
    for i in range(0, N):
        # left_up block
        i0 = i*P
        for p in range(P):
            A[i0+p, pixs[p, i]] = 1
        # right_up block
        A[i0:i0+P, 256+i] = -1
    # Z_mid constraint
    A[N*P, z_mid] = 1
    # left_bot block
    i0 = N*P+1-1
    A[i0:i0+254, :254] += np.eye(254)
    A[i0:i0+254, :255] += np.eye(255, k=1)[:254]*(-2)
    A[i0:i0+254, :256] += np.eye(256, k=2)[:254]
    # print(A[i0:i0+254, :256].shape)

    # vec b
    b = np.zeros(shape[0], dtype=float)
   
    b[:N*P] = np.tile(lndt, N)

    # Solve lsq
    x = np.array(np.linalg.lstsq(A, b, rcond=0.001)[0])
    # =====================================
    g = x[:256]
    e = np.exp(x[256:])
    return g, e


In [25]:
z_mid = (0+255)//2
# N(P-1) > 255
# takes 2x
P = len(bimgs)
n = 50 # sample number multiplier
N = int((255 / (P-1)) * n)
print(f"Z_mid = {z_mid}, P = {P}, N = {N}")
shape = (N*P+255, 256+N)

Z_mid = 127, P = 7, N = 2125


In [26]:
# [ch, P, h, w]
rgb_imgs = imgs.transpose([3, 0, 1, 2])
rgb_g = [SolveResponseCurve(img, P, N, z_mid)[0] for img in rgb_imgs]

Z_mid = 127, P = 7, N = 2125
Z_mid = 127, P = 7, N = 2125
Z_mid = 127, P = 7, N = 2125


In [27]:
w = lambda z: z-0+1 if z <= z_mid else 255-z+1 # add a positive value to prevent from divided-by-zero
w = np.vectorize(w)(np.arange(256))
# reverse energy function
# inv_f = lambda g, imgs: np.exp(np.sum([w[imgs[p]]*(g[imgs[p]]-lndt[p]) for p in range(P)], axis=0) / np.sum([w[imgs[p]] for p in range(P)], axis=0))
def inv_f(g, imgs):
    print("=====================")
    print(g.shape, g.min(), g.max())
    a = np.sum([w[imgs[p]]*(g[imgs[p]]-lndt[p]) for p in range(P)], axis=0)
    print(a.shape, a.min(), a.max())
    b = np.sum([w[imgs[p]] for p in range(P)], axis=0)
    print(b.shape, b.min(), b.max())
    res = np.exp(a/b)
    print(res.min(), res.max())
    return res



In [28]:
# [ch, p, w, h]
def normalize(img):
    _min = img.min()
    _max = img.max()
    return (img-_min)/(_max-_min)
# normalize = lambda i: (i-np.min(i))/(np.max(i)-np.min(i))
after = np.array([inv_f(g, imgs) for g, imgs in zip(rgb_g, rgb_imgs)])
after_norm = np.array([normalize(img)*255 for img in after]).astype('uint8')
after_norm = after_norm.transpose([1, 2, 0])

(256,) -1.22527254211553 6.43238256210402
(648, 972) -26.62660467865786 1721.7005118278219
(648, 972) 7 494
0.14551299183932714 752.63226494959
(256,) -1.5043692485938644 6.182108549708097
(648, 972) -32.66182564694557 1742.7414155183164
(648, 972) 7 504
0.11004790110594276 743.4229153128817
(256,) -1.3638758363946253 5.777900205843198
(648, 972) -32.72845648928875 1637.9013984591686
(648, 972) 7 501
0.12901801461834034 579.7112073643204


In [29]:
# show(np.concatenate([imgs[1], after_norm], axis=1))

# Save as EXR file

In [30]:
import OpenEXR as exr
import array

In [31]:
print(*after.shape[::-1][:2])
f = exr.OutputFile(f"output/{folder}.exr", exr.Header(*after.shape[::-1][:2]))
exr_img = [array.array('f', normalize(i).flatten()).tostring() for i in after]
f.writePixels({
    'R':exr_img[2],
    'G':exr_img[1],
    'B':exr_img[0],
})
f.close()

972 648
<ipython-input-31-4af13dc24cc0>:3: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  exr_img = [array.array('f', normalize(i).flatten()).tostring() for i in after]


In [32]:
cv2.imwrite(f'output/{folder}.hdr', after_norm)

True

In [33]:
import seaborn as sns
sns.set_theme(style="darkgrid")

def show_g(gs):
    plt.clf()
    color = ['blue', 'green', 'red']
    
    for i, g in enumerate(gs):
        sns.lineplot(data=g, color=color[i])
    

In [34]:
# show_g(rgb_g)